In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
import requests
import zipfile
from pathlib import Path

In [3]:
# For audio playback in notebook
def play_audio(path, sr=16000):
    audio, _ = librosa.load(path, sr=sr)
    return ipd.Audio(audio, rate=sr)


In [8]:
from datasets import load_dataset

dataset = load_dataset("seungheondoh/LP-MusicCaps-MSD", split="train")


(…)-00000-of-00002-557f11f5f8c72ffa.parquet:   0%|          | 0.00/128M [00:00<?, ?B/s]

(…)-00001-of-00002-841359b23dd9a178.parquet:   0%|          | 0.00/118M [00:00<?, ?B/s]

(…)-00000-of-00001-dcf74b4dfad2da31.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

(…)-00000-of-00001-bc348328eb4115ed.parquet:   0%|          | 0.00/20.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/444865 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/34631 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/34481 [00:00<?, ? examples/s]